# Deep Learning Project - Lisbon traffic sign experiment

In [1]:
import numpy as np
import pandas as pd
import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import os
import matplotlib.pyplot as plt

## Predicting test set images

In [3]:
# Load a saved model
model = keras.models.load_model('model_all_data_3ConvLayers_down_changed_architecture_100_100_1to15')

In [4]:
test_dir = '/Users/philippmetzger/Documents/GitHub/Deep_Learning_Project_Group_10/Data/lisbon_traffic_signs'
#test_dir = '/Users/philippmetzger/Documents/GitHub/Deep_Learning_Project_Group_10/Data/Meta'

### Load ground truth data

In [5]:
gt = pd.read_excel('/Users/philippmetzger/Documents/GitHub/Deep_Learning_Project_Group_10/Data/lisbon_traffic_signs/ground_truth.xlsx')
gt

,Index,Target
0,1,17
1,2,17
2,3,17
3,4,17
4,5,17
...,...,...
95,108,17
96,109,28
97,110,35
98,111,15


### Normalization

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(100,100),
    batch_size=20,
    shuffle=False,
    #class_mode=None,
    class_mode="categorical",
    color_mode="grayscale"
    )

x_test=np.concatenate([test_generator.next()[0] for i in range(test_generator.__len__())])

test_datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)

test_datagen.fit(x_test)

Found 100 images belonging to 1 classes.


##### Re-Creating the generators with the normalized image data

In [49]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(100, 100),
        batch_size=20,
        shuffle=False,
        color_mode="grayscale",
        class_mode=None)

test_generator.reset() 
pred_scores = model.predict_generator(test_generator)
class_pred = np.argmax(pred_scores, axis=-1)

Found 100 images belonging to 1 classes.


/Users/philippmetzger/anaconda3/envs/dl_pip_2/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [56]:
target = list(gt['Target'])

In [57]:
df = pd.DataFrame(data={'target':target, 'prediction':class_pred})
df

,target,prediction
0,17,17
1,17,17
2,17,17
3,17,17
4,17,17
...,...,...
95,17,17
96,28,11
97,35,25
98,15,17


In [58]:
i = 2
slice = [range(0,40), range(40,80), range(80,100)]

df.iloc[slice[i],:]

,target,prediction
80,35,13
81,17,13
82,17,11
83,25,17
84,13,13
85,17,23
86,17,35
87,33,28
88,17,1
89,13,13


In [60]:
bool_ = (class_pred == target)
bool_

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
       False,  True, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False,  True,
       False, False,  True, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False,  True, False, False, False, False,  True,
       False, False, False, False,  True,  True, False, False, False,
       False])

In [62]:
sum(bool_) / len(bool_)

0.36